In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

# df_books=df_books.drop_duplicates(subset=['title'])
# df_ratings=df_ratings.drop_duplicates(subset=['user','isbn'])

# PREPROCESSING DATA

In [ ]:
# select only those rows where each book has more than 100 ratings
v1=df_ratings.isbn.value_counts()
relevent_books=df_ratings[df_ratings.isbn.isin(v1[v1 >= 100].index)]
relevent_books=df_books.loc[df_books['isbn'].isin(relevent_books['isbn'])]


# select only those rows where each user has more than 200 ratings
v2=df_ratings.user.value_counts()
relevent_users=df_ratings[df_ratings.user.isin(v2[v2 >= 200].index)]
relevent_users=relevent_users.loc[relevent_users['isbn'].isin(relevent_books['isbn'])]

# using DataFrame.pivot as it can handle non-numerical data. no encoding needed for isbn
pivoted_ratings=relevent_users.pivot(values='rating',index='isbn',columns='user').fillna(0)
pivoted_ratings_csrmatrix=csr_matrix(pivoted_ratings)

# regular function

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

  book_list=[]
  recommended_books=[]
  recommended_books.append(book)

  if(book not in df_books['title'].values):
    print("Book "+" not found!")
    return ""

  req_isbn=df_books[(df_books['title'] == book)]
  req_isbn=req_isbn.iloc[0].isbn

  nbrs = NearestNeighbors(n_neighbors=6,metric='cosine').fit(pivoted_ratings_csrmatrix)
  distance, indices = nbrs.kneighbors(pivoted_ratings_csrmatrix)

  c=0
  for i in pivoted_ratings.index:
    if(i==req_isbn):
      break
    c+=1


  isbn_list=[]
  distance_list=[]

  for i in range(len(indices[c])):
    isbn_list.append(pivoted_ratings.iloc[indices[c][i]].name)
    distance_list.append(distance[c][i])

  isbn_list.pop(0)
  distance_list.pop(0)

  temp2=[]
  for i in range(len(isbn_list)):
    temp=[]
    temp.append(df_books[df_books['isbn']==isbn_list[i]].title.item())
    temp.append(distance_list[i])
    temp2.append(temp)

  temp2.reverse()
  recommended_books.append(temp2)

  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
